In [1]:
from openbabel import pybel

In [7]:
# Before doing anything, we define the paths to open the files 

smi_1 = "PATH1.smi"   # Query
smi_2 = "PATH2.smi"     # Search


In [8]:
# PART 1

# First of all, we define the Lipinksi rules

def lipinski(mol):
    desc = mol.calcdesc()
    return desc['HBD'] <= 5 and desc['HBA1'] <= 10 and desc['MW'] <= 500 and desc['logP'] <= 5

# Now we run each line (molecule) and if it obeys Lipinksi rules, it prints it.

for mol in pybel.readfile('smi',smi_2):
    if lipinski(mol) == True:
        print(mol.write('can').rstrip())


CN1CCN(CC1)Cc1ccc(cc1)C(=O)Nc1ccc(c(c1)Nc1nccc(n1)c1cccnc1)C	Imatinib
CCN(CCNC(=O)c1c(C)[nH]c(c1C)/C=C/1\C(=O)Nc2c1cc(F)cc2)CC	Sunitinib


In [25]:
# PART 2



# First of all, we define the Lipinksi rules (This is not needed since it is already stated in the first one, but just in case you want to do it from 0)

def lipinski(mol):
    desc = mol.calcdesc()
    return desc['HBD'] <= 5 and desc['HBA1'] <= 10 and desc['MW'] <= 500 and desc['logP'] <= 5

# Now we run each line (molecule) and if it obeys Lipinksi rules, it prints it.
# Also, we calculate the Tanimoto Score against the first molecule of the query file (smi_1)

counter = 0

for mol2 in pybel.readfile('smi',smi_1):
    if counter == 0:
        for mol in pybel.readfile('smi',smi_2):
            if lipinski(mol) == True:
                query_first_fp = mol2.calcfp()
                search_f = mol.calcfp()
                tanimoto = (search_f | query_first_fp)
                tanimoto = format(tanimoto,'.4f')
                print(str(mol.write('can').rstrip()) + "\t" + str(tanimoto))
        counter += 1
        

CN1CCN(CC1)Cc1ccc(cc1)C(=O)Nc1ccc(c(c1)Nc1nccc(n1)c1cccnc1)C	Imatinib	1.0000
CCN(CCNC(=O)c1c(C)[nH]c(c1C)/C=C/1\C(=O)Nc2c1cc(F)cc2)CC	Sunitinib	0.3390


In [49]:
smi_2 = "PATH3.smi" # Search

# Part 3

# This part is similar to the part 2, but in this case we need to also print the Tanimoto scores

# First of all, we define the Lipinksi rules (This is not needed since it is already stated in the first one and the second one, but just in case you want to do it from 0)

def lipinski(mol):
    desc = mol.calcdesc()
    return desc['HBD'] <= 5 and desc['HBA1'] <= 10 and desc['MW'] <= 500 and desc['logP'] <= 5

# In this case, we will also compare the molecules to the 1st query molecule.
# Also, it needs to match Lipinksi rules and should be shown in order of their Tanimoto Score, so:

counter = 0
dict_molecules = {}
list_values_tanimotos = []


for mol2 in pybel.readfile('smi',smi_1):
    if counter == 0:
        for mol in pybel.readfile('smi',smi_2):                  # THIS IS NOT THE SMI SHOWN IN THE PDF. YOU JUST NEED TO REPLACE THIS FOR THE FILE YOU NEED! THE 1k.smi!
            if lipinski(mol) == True:
                query_first_fp = mol2.calcfp()
                search_f = mol.calcfp()
                tanimoto_score = (search_f | query_first_fp)
                tanimoto_score = format(tanimoto_score,'.4f')
                mol_id = mol.write('can').rstrip()
                list_values_tanimotos.append(tanimoto_score)
                dict_molecules[tanimoto_score] = mol_id
        counter += 1
        
list_values_tanimotos.sort(reverse=True)

counter2 = 1
for value in list_values_tanimotos:
    for key in dict_molecules.keys():
        if value == key and counter2 <= 10:
            print (str(dict_molecules[key]) + "\t" + str(key))
            counter2+=1


O=C(NC(C)(C)C)CN(C(=O)c1cc(nc2c1cccc2)c1ccc(cc1)C)C	ZINC10293777	0.4337
CCc1ccc(cc1)C(=O)N1CC[NH+](CC1)[C@H](C(=O)Nc1ccc(cc1)C(=O)C)C	ZINC27743523	0.4194
O=C([C@@H]1COc2c(O1)cccc2)N1CCN(CC1)c1nc(nc2c1cccc2)c1cccnc1	ZINC13096382	0.4038
CCNC(=O)CN(c1nc(C)cc(c1C#N)C)CC	ZINC42370730	0.3943
O=C(c1ccc(cc1)CN1CC(=O)N2[C@H](C1=O)CCC2)Nc1ccccc1Cl	ZINC21526936	0.3895
Clc1ccc(cc1)C[NH+]1CCN(CC1)C(=O)CNC(=O)c1ccc(cc1)N1CCCC1=O	ZINC18017580	0.3865
C[C@@H]1OCCN(C1)Cc1ccc(cc1)C(=O)Nc1cccc2c1[nH]nc2	ZINC25090321	0.3842
CNC(=O)c1ccc(cc1)C[NH+](Cc1ccc(cc1)C(=O)Nc1ccccc1)C	ZINC07996780	0.3813
O=C(c1cccc(c1)CN1C(=O)CNC1=O)Nc1ccc(c(c1)C)n1cnnn1	ZINC29524869	0.3802
Fc1ccc(cc1)Nc1ncccc1C(=O)N(CC1OCCO1)C	ZINC39230173	0.3777


In [78]:
smi_1 = "PATH1.smi"   # Query
smi_2 = "PATH3.smi"       # Search

# Part 4

# This part is similar to the part 3, but we need to get rid of the "counter". We want to compare all queries against all searched molecules

# First of all, we define the Lipinksi rules (This is not needed since it is already stated in the first, second and third part. But just in case you want to do it from 0)

def lipinski(mol):
    desc = mol.calcdesc()
    return desc['HBD'] <= 5 and desc['HBA1'] <= 10 and desc['MW'] <= 500 and desc['logP'] <= 5

# In this case, we will also compare the molecules to the 1st query molecule.
# Also, it needs to match Lipinksi rules and should be shown in order of their Tanimoto Score, so:


dict_molecules = {}
list_values_tanimotos = []
sum_tanimoto = 0
average_tanimoto = 0

for mol2 in pybel.readfile('smi',smi_1):
    for mol in pybel.readfile('smi',smi_2):                  # THIS IS NOT THE SMI SHOWN IN THE PDF. YOU JUST NEED TO REPLACE THIS FOR THE FILE YOU NEED! THE 1k.smi!
        if lipinski(mol) == True:
            mol_id = mol.write('can').rstrip()
            if mol_id not in dict_molecules.keys():
                query_first_fp = mol2.calcfp()
                search_f = mol.calcfp()
                tanimoto_score = (search_f | query_first_fp)
                mol_id = mol.write('can').rstrip()
                list_values_tanimotos.append(tanimoto_score)
                dict_molecules[mol_id] = [tanimoto_score]
            elif mol_id in dict_molecules.keys():
                query_first_fp = mol2.calcfp()
                search_f = mol.calcfp()
                tanimoto_score = (search_f | query_first_fp)
                dict_molecules[mol_id] += [tanimoto_score]
                
dict_molecules_average_tanimotos = {}
sum_tanimotos = 0
counter = 0

for listanimotos in dict_molecules.keys():
    for item in dict_molecules[listanimotos]:
        sum_tanimotos += item
        counter+=1
    average_tanimoto = (sum_tanimotos/counter)
    dict_molecules_average_tanimotos[listanimotos] = format(average_tanimoto, ".4f")
    average_tanimoto = 0
    sum_tanimotos = 0
    counter = 0
        
counter2 = 1

sorted_values = sorted(dict_molecules_average_tanimotos.values(), reverse=True)

for value in sorted_values:
    if counter2 <= 10:
        for key in dict_molecules_average_tanimotos.keys():
            if value == dict_molecules_average_tanimotos[key]:
                print(str(key)+'\t'+str(dict_molecules_average_tanimotos[key]))
                counter2+=1
    
        
        



Cc1cccc(n1)n1cnc(c1)C(=O)N1CCN(CC1)c1cccc(c1)C(F)(F)F	ZINC01383835	0.3744
O=C(c1cc(nc2c1cccc2)C(C)C)NCc1ccc(nc1)n1cccn1	ZINC39548744	0.3418
O=C([C@@H]1COc2c(O1)cccc2)N1CCN(CC1)c1nc(nc2c1cccc2)c1cccnc1	ZINC13096382	0.3412
COC(=O)c1ccc(cc1)NC(=O)c1cn(C)c2c(c1=O)cc(cc2)Cl	ZINC41056662	0.3394
O=C(c1cccc(c1)CN1C(=O)CNC1=O)Nc1ccc(c(c1)C)n1cnnn1	ZINC29524869	0.3375
CCOC(=O)Cc1ncccc1C(=O)Nc1ccc(cc1)I	ZINC29108003	0.3355
O=C1CCc2c(N1)ccc(c2)[C@@H](NC(=O)C(=O)Nc1cccc(c1)C(=O)N(C)C)C	ZINC41926777	0.3343
O=C(NC(C)(C)C)CN(C(=O)c1cc(nc2c1cccc2)c1ccc(cc1)C)C	ZINC10293777	0.3340
CCc1ccc(cc1)C(=O)N1CC[NH+](CC1)[C@H](C(=O)Nc1ccc(cc1)C(=O)C)C	ZINC27743523	0.3268
CCNC(=O)CN(c1nc(C)cc(c1C#N)C)CC	ZINC42370730	0.3254


7
